<a href="https://colab.research.google.com/github/atharva0300/BE-8th-Semester/blob/main/nlp_mini_project/news_article_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [300]:
pip install newspaper3k

In [301]:
url = input("Enter the url of the news article webpage : \n")

Enter the url of the news article webpage : 
https://economictimes.indiatimes.com/markets/cryptocurrency/bitcoin-tops-70000-again-after-slumping-on-us-etf-outflows/articleshow/108773929.cms


In [302]:
# obtain the news paper
from newspaper import Article

def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

article_text = extract_article(url)
print(article_text)






(You can now subscribe to our

(You can now subscribe to our ETMarkets WhatsApp channel

New York: Bitcoin enthusiasts appear to be shrugging off last week's outflows from US exchange-traded funds, with the largest cryptocurrency briefly climbing back above $70,000 again.Most digital assets were higher Monday, with Bitcoin gaining as much as 5.8% to $70,014. That's the first time the token has been above $70,000 in more than a week. Ether was up around 5%, while Solana and Dogecoin were both more than 4% higher.Almost $900 million was pulled from those ETFs last week, reflecting continual outflows from the Grayscale Bitcoin Trust as well as a moderation in subscriptions for offerings from BlackRock Inc. and Fidelity Investment. The group of 10 funds saw one of the worst weeks of the year since they were launched in January."Even though ETF inflows have hit a drag, order books are loaded on the bid side around the 60k area, showing that the market is eager to buy the dip," said Nath

In [303]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize each sentence into words, remove stopwords, and perform stemming
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    preprocessed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [stemmer.stem(word) for word in words if word not in stop_words]
        preprocessed_sentences.append(" ".join(words))

    return preprocessed_sentences

def textrank_summarizer(text, num_sentences=3):
    # Preprocess the text
    preprocessed_sentences = preprocess_text(text)

    # Create TF-IDF matrix
    tfidf = TfidfVectorizer().fit_transform(preprocessed_sentences)

    # Calculate similarity matrix
    similarity_matrix = cosine_similarity(tfidf, tfidf)

    # Create graph using similarity matrix
    graph = nx.from_numpy_array(similarity_matrix)

    # Calculate PageRank scores
    scores = nx.pagerank(graph)

    # Sort sentences by PageRank scores
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(preprocessed_sentences)), reverse=True)

    # Get top N sentences for summary
    summary_sentences = [sentence for score, sentence in ranked_sentences[:num_sentences]]

    # Reorder the summary sentences based on their order in the original text
    summary = " ".join([preprocessed_sentences[preprocessed_sentences.index(sent)] for sent in summary_sentences])

    return summary



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [304]:
# Output of Textrank algorithm

summary = textrank_summarizer(article_text)
summary_list = summary.split(' ')
output_text = ""
for i in range(len(summary_list)) :
  if(i%30==0 and i!=0 ) :
    print('\n')
    output_text += "\n"
  else :
    output_text += summary_list[i] + " "
    print(summary_list[i] , end = " ")

textrank_output = output_text

( subscrib ( subscrib etmarket whatsapp channel new york : bitcoin enthusiast appear shrug last week 's outflow us exchange-trad fund , largest cryptocurr briefli climb back $ 70,000 again.most 

asset higher monday , bitcoin gain much 5.8 % $ 70,014 . 's first time token $ 70,000 week . ether around 5 % , solana dogecoin 4 % 

$ 900 million pull etf last week , reflect continu outflow grayscal bitcoin trust well moder subscript offer blackrock inc. fidel invest . 

In [305]:
pip install sumy

In [306]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# Function to summarize a news article using LexRank
def summarize_news_article(text, language='english', sentences_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer(language))
    summarizer = LexRankSummarizer()

    # Summarize the article
    summary = summarizer(parser.document, sentences_count)
    summary_text = ' '.join([str(sentence) for sentence in summary])

    return summary_text

# Summarize the news article
summary = summarize_news_article(article_text)
summary_list = summary.split(' ')
output_text = ""
for i in range(len(summary_list)) :
  if(i%30==0 and i!=0 ) :
    print('\n')
    output_text += "\n"
  else :
    output_text += summary_list[i] + " "
    print(summary_list[i] , end = " ")

lexrank_output = output_text


(You can now subscribe to our (You can now subscribe to our ETMarkets WhatsApp channel New York: Bitcoin enthusiasts appear to be shrugging off last week's outflows from US exchange-traded 

with the largest cryptocurrency briefly climbing back above $70,000 again.Most digital assets were higher Monday, with Bitcoin gaining as much as 5.8% to $70,014. 

In [307]:
pip install transformers


In [308]:
from transformers import pipeline

# Load the summarization pipeline
#summarizer = pipeline("summarization")

In [309]:
# formatting the text
import textwrap
formatted_text = textwrap.fill(article_text, width=80)
formatted_text

'    (You can now subscribe to our  (You can now subscribe to our ETMarkets\nWhatsApp channel  New York: Bitcoin enthusiasts appear to be shrugging off last\nweek\'s outflows from US exchange-traded funds, with the largest cryptocurrency\nbriefly climbing back above $70,000 again.Most digital assets were higher\nMonday, with Bitcoin gaining as much as 5.8% to $70,014. That\'s the first time\nthe token has been above $70,000 in more than a week. Ether was up around 5%,\nwhile Solana and Dogecoin were both more than 4% higher.Almost $900 million was\npulled from those ETFs last week, reflecting continual outflows from the\nGrayscale Bitcoin Trust as well as a moderation in subscriptions for offerings\nfrom BlackRock Inc. and Fidelity Investment. The group of 10 funds saw one of\nthe worst weeks of the year since they were launched in January."Even though ETF\ninflows have hit a drag, order books are loaded on the bid side around the 60k\narea, showing that the market is eager to buy the 

In [310]:
from transformers import BartTokenizer, BartForConditionalGeneration
import textwrap

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Wrap the formatted text to 1024 tokens per input (maximum input length for BART)
wrapped_text = textwrap.fill(article_text, width=1024)

# Tokenize the text
inputs = tokenizer(wrapped_text, return_tensors='pt', max_length=1024, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [311]:
# Output of BART Transformer

summary_list = summary.split(' ')
output_text = ""

for i in range(len(summary_list)) :
  if(i%30==0 and i!=0 ) :
    print('\n')
    output_text += "\n"
  else :
    output_text += summary_list[i] + " "
    print(summary_list[i] , end = " ")

bart_output = output_text

Most digital assets were higher Monday, with Bitcoin gaining as much as 5.8%. Ether was up around 5%, while Solana and Dogecoin were both more than 4% higher. Almost $900 

was pulled from those ETFs last week. 

In [312]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the input text
inputs = tokenizer.encode("summarize: " + article_text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [313]:
# Output of T5 Transformer

summary_list = summary.split(' ')
output_text = ""
for i in range(len(summary_list)) :
  if(i%30==0 and i!=0 ) :
    print('\n')
    output_text += "\n"
  else :
    output_text += summary_list[i] + " "
    print(summary_list[i] , end = " ")

t5_output = output_text

the largest cryptocurrency has been above $70,000 in more than a week. it's the first time the token has been above $70,000 in more than a week. it's the first 

the token has been above $70,000 in more than a week. 

In [314]:
pip install reportlab

In [315]:
# Create a pdf and store the outputs in the pdf file

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from google.colab import files

# Create a PDF file
def create_pdf(file_name, text):
    custom_page_height = 1600  # Custom page height in points
    c = canvas.Canvas(file_name,  pagesize=(letter[0], custom_page_height))
    lines = text.split('\n')
    y_position = 1550  # Starting vertical position
    line_height = 12  # Height of each line
    max_width = 500  # Maximum width of the text

    for line in lines:
        # Calculate the height of the text
        text_width = c.stringWidth(line, "Helvetica", 12)

        # If the text exceeds the maximum width, create a new line
        if text_width > max_width:
            parts = [line[i:i+100] for i in range(0, len(line), 100)]  # Split line into parts
            for part in parts:
                c.drawString(20, y_position, part)
                y_position -= line_height  # Move to the next line
        else:
            c.drawString(20, y_position, line)
            y_position -= line_height  # Move to the next line

    c.save()


# Text to write in the PDF
text = ""
text = "                                                                News Summarizer Report\n\n\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += "Article\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += article_text
text += "\n\n\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += "TextRank Algorithm Output\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += textrank_output
text += "\n\n\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += "LexRank Algorithm Output\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += lexrank_output
text += "\n\n\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += "BART Output\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += bart_output
text += "\n\n\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += "T5 Output\n"
text += "------------------------------------------------------------------------------------------------------------------------\n"
text += t5_output
text += "\n\n\n"

# Create and save the PDF file
file_name = "News_Summarizer_Report.pdf"
create_pdf(file_name, text)

# Download the PDF file
files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>